This is a quick rundown of how to run LESSPayne by writing python code.

The script LESSPayne/LESSPayne/cli/run.py allows you to do this from the command line

In [1]:
import yaml

# Imports for automatic running
from LESSPayne.PayneEchelle.run_payne_echelle import run_payne_echelle
from LESSPayne.autosmh.run_normalization import run_normalization
from LESSPayne.autosmh.run_eqw_fit import run_eqw_fit
from LESSPayne.autosmh.run_stellar_parameters import run_stellar_parameters
from LESSPayne.autosmh.run_synth_fit import run_synth_fit

## Restart and Run All to get the total runtime to fit this spectrum
import time
startall = time.time()

In [2]:
## This is an example configuration file for LESSPayne, parsed with yaml

## What is generally recommended is to put this in a file and load it
## But for this tutorial we'll put it in a string
cfg_string = """
# These directories are automatically created
output_directory: ./outputs
figure_directory: ./figs
# This is used to set filenames, make it unique for each object
output_name: tutorial-hd122563
# TODO Specify the spectrum filenames
# If there's only one file, just do one row, but don't remove the dash (-) or it may break
# Make sure the first one contains your autovelocity wavelength range
spectrum_fnames:
- ./hd122563red_multi.fits
- ./hd122563blue_multi.fits
# Note: right now most code will ignore this flag and always overwrite
# To be updated later
overwrite_files: True
payne_fname: tutorial-hd122563_paynefit.npz
smh_fname: tutorial-hd122563_lesspayne.smh
# TODO update this to point to the actual file
NN_file: ./NN_normalized_spectra_float16_fixwave.npz
# parameters for automatic velocity finding
autovelocity:
  # TODO update this to point to the actual file
  template_spectrum_fname: hd122563.fits
  wavelength_region_min: 5150
  wavelength_region_max: 5200
# parameters for automatic payne fit
payne:
  # automatically find the velocity from xcor
  initial_velocity: null
  # restrict the wavelength range
  # the full allowed range is 3500-10000A
  wmin: 4800
  wmax: 7000
  mask_list:
  - - 6276
    - 6320
  - - 6866
    - 6881
  - - 6883
    - 6962
  - - 6985
    - 7070
  rv_target_wavelengths:
  - 5183
  - 4861
  initial_parameters:
    Teff: 4500
    logg: 1.5
    MH:   -2.0
    aFe:  0.4
  save_figures: False
run_normalization:
  # threshold for deciding whether to mask a line in the Payne model
  mask_sigma: 4.0
  # parameters for growing the mask size a bit
  mask_smooth: 2
  mask_thresh: 0.15
  # maximum fraction of masked pixels per order; increases mask_sigma by 0.5 until reached
  max_mask_frac: 0.8
  # minimum fraction of unmasked pixels/knot; increases knot_spacing by 2 until reached
  min_frac_per_knot: 0.05
  # blue and red trim in pixels
  blue_trim: 30
  red_trim: 30
  # normalization kwargs
  continuum_spline_order: 3
  continuum_max_iterations: 5
  # Flag to output a normalization figure
  save_figure: True
run_eqw_fit:
  max_fwhm: 1.0
  # TODO update this to point to the right filename
  eqw_linelist_fname: rpa2k.eqwshort-moog.txt
  extra_eqw_linelist_fname: null
  # threshold for deciding whether to mask a line in the Payne model
  mask_sigma: 2.0
  # parameters for growing the mask size a bit
  mask_smooth: 2
  mask_thresh: 0.15
  clear_all_existing_fits: True
  save_figure: False
run_stellar_parameters:
  method: rpa_calibration
  manual_Teff: null
  manual_logg: null
  manual_vt: null
  manual_MH: null
  manual_aFe: 0.4
  measure_eqw_abundances: True
  save_figure_eqw: True
run_synth_fit:
  max_fwhm: 1.0
  # Note: check the paths in the master list to make sure they're right!
  # TODO update this list
  synthesis_linelist_fname: quick.synth-master-v1.txt
  #synthesis_linelist_fname: null
  extra_synthesis_linelist_fname: null
  # number of times to iterate through fitting all syntheses
  # the actual number of iterations is 2x this: the first set puts a smoothing prior,
  # while the second set uses known abundances
  # the runtime is essentially proportional to this x the number of synthesis lines
  num_iter_all: 2
  # each optimization takes a few times to minimize the chi2
  max_iter_each: 3
  # parameters for initial prior on smoothing in Angstroms
  smooth_approx: 0.1
  smooth_scale : 0.3
  # Set O, Si, Ca, Ti to [Mg/Fe]: currently forced to be true, capped at +/- 0.4
  # use_MgFe_for_aFe: True
  # Set neutron-capture elements to r-process value: currently forced to be true
  # use_EuFe_for_rproc: True
  clear_all_existing_syntheses: False
  save_figure: True
# Not used yet
run_errors:
  calculate_stellar_params_spectroscopic_uncertainties: False
  e_Teff: 50
  e_logg: 0.1
  e_vt: 0.1
  e_MH: 0.1
  save_figure: True
# Not used yet
run_plot:
"""

In [3]:
cfg = yaml.load(cfg_string, yaml.FullLoader)

In [4]:
## This parses into a dictionary as you can see
cfg

{'output_directory': './outputs',
 'figure_directory': './figs',
 'output_name': 'tutorial-hd122563',
 'spectrum_fnames': ['./hd122563red_multi.fits', './hd122563blue_multi.fits'],
 'overwrite_files': True,
 'payne_fname': 'tutorial-hd122563_paynefit.npz',
 'smh_fname': 'tutorial-hd122563_lesspayne.smh',
 'NN_file': './NN_normalized_spectra_float16_fixwave.npz',
 'autovelocity': {'template_spectrum_fname': 'hd122563.fits',
  'wavelength_region_min': 5150,
  'wavelength_region_max': 5200},
 'payne': {'initial_velocity': None,
  'wmin': 4800,
  'wmax': 7000,
  'mask_list': [[6276, 6320], [6866, 6881], [6883, 6962], [6985, 7070]],
  'rv_target_wavelengths': [5183, 4861],
  'initial_parameters': {'Teff': 4500, 'logg': 1.5, 'MH': -2.0, 'aFe': 0.4},
  'save_figures': False},
 'run_normalization': {'mask_sigma': 4.0,
  'mask_smooth': 2,
  'mask_thresh': 0.15,
  'max_mask_frac': 0.8,
  'min_frac_per_knot': 0.05,
  'blue_trim': 30,
  'red_trim': 30,
  'continuum_spline_order': 3,
  'continuum_m

In [5]:
# run PayneEchelle. Creates outputs/tutorial-hd122563_paynefit.npz and some figures
run_payne_echelle(cfg)

2023-01-09 14:15:20,272 [INFO    ] Recognized CarPy product. Using zero-indexed flux/noise extensions (bands) 1/2


Saving to output directory: ./outputs ./outputs/tutorial-hd122563_paynefit.npz
Saving figures to output directory: ./figs
{'initial_velocity': None, 'wmin': 4800, 'wmax': 7000, 'mask_list': [[6276, 6320], [6866, 6881], [6883, 6962], [6985, 7070]], 'rv_target_wavelengths': [5183, 4861], 'initial_parameters': {'Teff': 4500, 'logg': 1.5, 'MH': -2.0, 'aFe': 0.4}, 'save_figures': False}
Automatic velocity with template hd122563.fits


2023-01-09 14:15:20,778 [INFO    ] Recognized CarPy product. Using zero-indexed flux/noise extensions (bands) 1/2
INFO:LESSPayne.specutils.spectrum:Recognized CarPy product. Using zero-indexed flux/noise extensions (bands) 1/2


Automatic velocity from 5150-5200: -52.8 (+ 27.8 for vhel)
Keeping 21/34 orders between 4800-7000
Keeping 21/34 orders between 4800-7000
Median RV order wavelength: {np.median(wavelength[iorder]):.1f}
Running with PayneEchelle (./NN_normalized_spectra_float16_fixwave.npz)
starting fit
Pre Fit: Finding the best radial velocity initialization
1 / 1
Pre Fit: Fitting the blaze-normalized spectrum
1 / 1
Pre Fit: Finding the best continuum initialization
Final Fit: Fitting the whole spectrum with all parameters simultaneously
p0: Teff=4616 logg=1.44 FeH=-2.89 aFe=0.31
  chunk 0-20 rv=-53.2 vbroad=2.1
1 / 1
PayneEchelle Fit Took 84.0
[Teff [K], logg, Fe/H, Alpha/Fe] =  4640.0 1.52 -2.81 0.33
vbroad [km/s] =  2.2
RV [km/s] =  -53.1
Chi square =  11.845178354641858
TOTAL TIME: 85.4


In [6]:
# create SMHR file with normalizations.
# Uses PayneEchelle model to find pixels to mask
# Creates ./outputs/tutorial-hd122563_lesspayne.smh, overwrites existing files
# This file can be opened in a GUI:
#  go into LESSPayne/LESSPayne/smh/gui and run `pythonw __main__.py`
# Also creates a figure in ./figs/ showing all the normalizations
run_normalization(cfg)

2023-01-09 14:16:45,696 [INFO    ] Recognized CarPy product. Using zero-indexed flux/noise extensions (bands) 1/2
INFO:LESSPayne.specutils.spectrum:Recognized CarPy product. Using zero-indexed flux/noise extensions (bands) 1/2
2023-01-09 14:16:45,725 [INFO    ] Recognized CarPy product. Using zero-indexed flux/noise extensions (bands) 1/2
INFO:LESSPayne.specutils.spectrum:Recognized CarPy product. Using zero-indexed flux/noise extensions (bands) 1/2


Saving to output directory: ./outputs
Saving figures to output directory: ./figs


Correcting at rv=-53.2
Total to load in data: 0.4
norm params [-0.32447051 -0.1942757  -0.27362286  0.17085493  1.          0.
  0.          0.          0.          0.          0.          2.23841138
 -0.53183321]
Wave=3362 SNR=9.2
Model evaluation exception at wave=3326-3399, no masks
A value in x_new is below the interpolation range.
  num_mask=0/2048
Wave=3389 SNR=12.9
Model evaluation exception at wave=3353-3426, no masks
A value in x_new is below the interpolation range.
  num_mask=0/2048
Wave=3421 SNR=16.1
Model evaluation exception at wave=3385-3458, no masks
A value in x_new is below the interpolation range.
  num_mask=0/2048
Wave=3455 SNR=19.7
Model evaluation exception at wave=3418-3491, no masks
A value in x_new is below the interpolation range.
  num_mask=0/2048
Wave=3488 SNR=23.2
Model evaluation exception at wave=3451-3525, no masks
A value in x_new is below the interpolation range.
  num_mask=0/2048
Wave=3523 SNR=27.7
Model evaluation exception at wave=3486-3560, no mask

Wave=5536
Wave=5627
Wave=5721
Wave=5818
Wave=5918
Wave=6022
Wave=6130
Wave=6242
Wave=6357
Wave=6477
Wave=6602
Wave=6732
Wave=6866
Wave=7007
Wave=7153
Wave=7305
Wave=7464
Wave=7630
Wave=7803
Wave=7985
Wave=8175
Wave=8374
  0.01 < 0.05 increasing knot_spacing to 27
  0.02 < 0.05 increasing knot_spacing to 29
  0.02 < 0.05 increasing knot_spacing to 31
  0.01 < 0.05 increasing knot_spacing to 33
  0.02 < 0.05 increasing knot_spacing to 35
  0.02 < 0.05 increasing knot_spacing to 37
  0.03 < 0.05 increasing knot_spacing to 39
  0.03 < 0.05 increasing knot_spacing to 41
  0.02 < 0.05 increasing knot_spacing to 43
  0.02 < 0.05 increasing knot_spacing to 45
Wave=8584
Wave=8804
  0.02 < 0.05 increasing knot_spacing to 27
  0.02 < 0.05 increasing knot_spacing to 29
Could not succeed in checking knots, stopping at it=3
attempt to get argmin of an empty sequence
Could not succeed in checking knots, stopping at it=4
attempt to get argmin of an empty sequence
Could not succeed in checking knots, s

/Users/alexji/lib/LESSPayne/LESSPayne/specutils/spectrum.py:77: RuntimeWarning: Mean of empty slice
  ivar = np.ones_like(flux) * 1.0/np.nanmean(flux)


Total time to run all: 9.6
Saving figure to ./figs/tutorial-hd122563_norm.png


Time to save figure: 4.1


In [7]:
# fit EQWs
# Reads ./outputs/tutorial-hd122563_lesspayne.smh and overwrites this file when it completes
run_eqw_fit(cfg)

2023-01-09 14:17:00,860 [INFO    ] Recognized CarPy product. Using zero-indexed flux/noise extensions (bands) 1/2
INFO:LESSPayne.specutils.spectrum:Recognized CarPy product. Using zero-indexed flux/noise extensions (bands) 1/2
2023-01-09 14:17:00,873 [INFO    ] Recognized CarPy product. Using zero-indexed flux/noise extensions (bands) 1/2
INFO:LESSPayne.specutils.spectrum:Recognized CarPy product. Using zero-indexed flux/noise extensions (bands) 1/2


Saving to output directory: ./outputs
Saving figures to output directory: ./figs
Time to add masks to normalizations and import models: 2.2
Total time to run all: 3.8


In [8]:
# put in stellar parameters and (optionally) measure EQW abundances
# overwrites the SMH file
run_stellar_parameters(cfg)

2023-01-09 14:17:04,642 [INFO    ] Recognized CarPy product. Using zero-indexed flux/noise extensions (bands) 1/2
INFO:LESSPayne.specutils.spectrum:Recognized CarPy product. Using zero-indexed flux/noise extensions (bands) 1/2
2023-01-09 14:17:04,650 [INFO    ] Recognized CarPy product. Using zero-indexed flux/noise extensions (bands) 1/2
INFO:LESSPayne.specutils.spectrum:Recognized CarPy product. Using zero-indexed flux/noise extensions (bands) 1/2


Saving to output directory: ./outputs
Saving figures to output directory: ./figs
Setting aFe=0.4
Final stellar parameters: T/g/v/M/a = 4733/1.82/1.75/-2.67/0.40
Measuring EQW abundances


/Users/alexji/lib/LESSPayne/LESSPayne/smh/photospheres/castelli_kurucz.py:53: UserWarning: Fourth stellar param is [alpha/Fe] = 0.4
  warnings.warn(
/Users/alexji/lib/LESSPayne/LESSPayne/smh/photospheres/castelli_kurucz.py:53: UserWarning: Fourth stellar param is [alpha/Fe] = 0.4
  warnings.warn(


Total time run_stellar_parameters: 3.1


Time to save eqw figure: 2.0


In [9]:
# import and iteratively fit syntheses
# overwrites the SMH file
run_synth_fit(cfg)

2023-01-09 14:17:09,735 [INFO    ] Recognized CarPy product. Using zero-indexed flux/noise extensions (bands) 1/2
INFO:LESSPayne.specutils.spectrum:Recognized CarPy product. Using zero-indexed flux/noise extensions (bands) 1/2
2023-01-09 14:17:09,743 [INFO    ] Recognized CarPy product. Using zero-indexed flux/noise extensions (bands) 1/2
INFO:LESSPayne.specutils.spectrum:Recognized CarPy product. Using zero-indexed flux/noise extensions (bands) 1/2


Saving to output directory: ./outputs
Saving figures to output directory: ./figs


/Users/alexji/lib/LESSPayne/LESSPayne/smh/photospheres/castelli_kurucz.py:53: UserWarning: Fourth stellar param is [alpha/Fe] = 0.4
  warnings.warn(
/Users/alexji/lib/LESSPayne/LESSPayne/smh/photospheres/castelli_kurucz.py:53: UserWarning: Fourth stellar param is [alpha/Fe] = 0.4
  warnings.warn(


Importing quick.synth-master-v1.txt
type
----
 syn
 syn
 syn
 syn
['syn' 'syn' 'syn' 'syn']
======Running synthesis iter 1=============
Fitting 4215.524 [[38.1]]


/Users/alexji/lib/LESSPayne/LESSPayne/smh/photospheres/castelli_kurucz.py:53: UserWarning: Fourth stellar param is [alpha/Fe] = 0.4
  warnings.warn(
/Users/alexji/lib/LESSPayne/LESSPayne/smh/photospheres/castelli_kurucz.py:53: UserWarning: Fourth stellar param is [alpha/Fe] = 0.4
  warnings.warn(
/Users/alexji/lib/LESSPayne/LESSPayne/smh/photospheres/castelli_kurucz.py:53: UserWarning: Fourth stellar param is [alpha/Fe] = 0.4
  warnings.warn(
/Users/alexji/lib/LESSPayne/LESSPayne/smh/photospheres/castelli_kurucz.py:53: UserWarning: Fourth stellar param is [alpha/Fe] = 0.4
  warnings.warn(
/Users/alexji/lib/LESSPayne/LESSPayne/smh/photospheres/castelli_kurucz.py:53: UserWarning: Fourth stellar param is [alpha/Fe] = 0.4
  warnings.warn(
/Users/alexji/lib/LESSPayne/LESSPayne/smh/photospheres/castelli_kurucz.py:53: UserWarning: Fourth stellar param is [alpha/Fe] = 0.4
  warnings.warn(
/Users/alexji/lib/LESSPayne/LESSPayne/smh/photospheres/castelli_kurucz.py:53: UserWarning: Fourth stellar 

...

Fitting 4554.029 [[56.1]]


/Users/alexji/lib/LESSPayne/LESSPayne/smh/photospheres/castelli_kurucz.py:53: UserWarning: Fourth stellar param is [alpha/Fe] = 0.4
  warnings.warn(
/Users/alexji/lib/LESSPayne/LESSPayne/smh/photospheres/castelli_kurucz.py:53: UserWarning: Fourth stellar param is [alpha/Fe] = 0.4
  warnings.warn(
/Users/alexji/lib/LESSPayne/LESSPayne/smh/photospheres/castelli_kurucz.py:53: UserWarning: Fourth stellar param is [alpha/Fe] = 0.4
  warnings.warn(
/Users/alexji/lib/LESSPayne/LESSPayne/smh/photospheres/castelli_kurucz.py:53: UserWarning: Fourth stellar param is [alpha/Fe] = 0.4
  warnings.warn(


Fitting 4129.72 [[63.1]]


/Users/alexji/lib/LESSPayne/LESSPayne/smh/photospheres/castelli_kurucz.py:53: UserWarning: Fourth stellar param is [alpha/Fe] = 0.4
  warnings.warn(
/Users/alexji/lib/LESSPayne/LESSPayne/smh/photospheres/castelli_kurucz.py:53: UserWarning: Fourth stellar param is [alpha/Fe] = 0.4
  warnings.warn(
/Users/alexji/lib/LESSPayne/LESSPayne/smh/photospheres/castelli_kurucz.py:53: UserWarning: Fourth stellar param is [alpha/Fe] = 0.4
  warnings.warn(
/Users/alexji/lib/LESSPayne/LESSPayne/smh/photospheres/castelli_kurucz.py:53: UserWarning: Fourth stellar param is [alpha/Fe] = 0.4
  warnings.warn(
/Users/alexji/lib/LESSPayne/LESSPayne/smh/photospheres/castelli_kurucz.py:53: UserWarning: Fourth stellar param is [alpha/Fe] = 0.4
  warnings.warn(
/Users/alexji/lib/LESSPayne/LESSPayne/smh/photospheres/castelli_kurucz.py:53: UserWarning: Fourth stellar param is [alpha/Fe] = 0.4
  warnings.warn(
/Users/alexji/lib/LESSPayne/LESSPayne/smh/photospheres/castelli_kurucz.py:53: UserWarning: Fourth stellar 

Total time run_synth_fit: 78.5


In [10]:
print(f"If restart/runall, this took {time.time()-startall:.1f} seconds")

If restart/runall, this took 187.9 seconds


At the end of this is an SMHR file in outputs/tutorial-hd122563_lesspayne.smh.

In general it is wise to do this in stages:
* run PayneEchelle (2-5 minutes), and optionally inspect the automatic fit to make sure it's OK
* run normalization. Once this is done, inspect the SMH file and make any manual modifications to normalization.
* run eqw fits. When this is done, inspect the SMH file and make any manual changes to the EQW fits.
* run stellar parameters. Right now the only options are to manually specify the parameters, or to use a calibrated version of the PayneEchelle parameters (calibrated to RPA Duplicates).
* once you are happy with EQW fits and stellar parameters, run synthesis fits. If running a lot of syntheses, this is the most time consuming part as it needs to iterate multiple times.

After the normalizations, you can always edit the SMH file manually to fix anything that didn't automatically work.